In [1]:
!pip install tensorflow tensorflow-metal opencv-python mediapipe scikit-learn matplotlib

Installation test for MacOS

In [1]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

2024-01-28 00:16:26.846449: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-01-28 00:16:26.846473: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-01-28 00:16:26.846476: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-01-28 00:16:26.846718: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-28 00:16:26.847132: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2024-01-28 00:16:30.320117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


112/782 [===>..........................] - ETA: 41s - loss: 5.2436 - accuracy: 0.0364

KeyboardInterrupt: 

In [16]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [22]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 

while cap.isOpened():

    # Read feed
    ret, frame = cap.read()

    # Show to screen
    cv2.imshow('OpenCV Feed', frame)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()